In [1]:
import numpy as np
import pandas as pd

# Data Analysis Using MongoDB

## Connect to MongoDB Database

In [2]:
from pymongo import MongoClient

# Create a client
transaction = MongoClient('mongodb://localhost:27017/')

# Connect to your database
db = transaction['Coinut']

## Fetch Data

In [3]:
# Select the collection from the database
collection = db['transaction']

# Fetch all doc data
data1 = collection.find()

In [4]:
# Define the index to currency mapping
currency_index = {
    0: 'BTC', 1: 'USD', 2: 'LTC', 3: 'ETH', 4: 'ETC', 5: 'USDT', 6: 'XMR', 7: 'BCH', 8: 'BTG', 9: 'B2X', 
    10: 'SBTC', 11: 'SGD', 12: 'CAD', 13: 'EUR', 14: 'NTD', 15: 'JPY', 16: 'GBP', 17: 'LCH', 18: 'CHF', 
    19: 'ZEC', 20: 'ADA', 21: 'MYR', 22: 'DAI', 23: 'XSGD', 24: 'XIDR'}

## Calculate Transaction Paires and amount

In [5]:
import requests

# API endpoint URL
url = 'https://api.coinut.com/spot'

# Send a GET request to the API
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Get the JSON data from the response
    data = response.json()
    
    # Process the data as needed
    print(data)
else:
    # If the request was not successful, print the status code and error message
    print(f"Request failed with status code {response.status_code}: {response.text}")

[{'asset': 'LTCBTC', 'high24': '0.00301000', 'highest_buy': '0.00295000', 'inst_id': 1, 'last': '0.00301000', 'low24': '0.00299000', 'lowest_sell': '0.00296000', 'prev_trans_id': 33173893127, 'price_change_24': '0.00668896', 'timestamp': 1686543566315212, 'trans_id': 33173906306, 'volume': '592480.14428643', 'volume24': '0.16500000', 'volume24_quote': '0.00049499', 'volume_quote': '8351.41267832'}, {'asset': 'ETHBTC', 'high24': '0.06852000', 'highest_buy': '0.06722000', 'inst_id': 2, 'last': '0.06852000', 'low24': '0.06852000', 'lowest_sell': '0.06739000', 'prev_trans_id': 33173883404, 'price_change_24': '0', 'timestamp': 1686543505217517, 'trans_id': 33173883408, 'volume': '42684.85627851', 'volume24': '0.06000000', 'volume24_quote': '0.00411120', 'volume_quote': '2186.65571762'}, {'asset': 'ETCBTC', 'high24': '0.00072100', 'highest_buy': '0.00057700', 'inst_id': 3, 'last': '0.00071800', 'low24': '0.00071800', 'lowest_sell': '0.00058000', 'prev_trans_id': 33173884917, 'price_change_24

In [6]:
# Initialize an empty dictionary to store the values
pair_dic = {}

# Iterate over the data
for item in data:
    asset = item['asset']
    inst_id = item['inst_id']
    
    # Add the values to the dictionary with asset as the key
    pair_dic[asset] = inst_id

# Print the result
print(pair_dic)

{'LTCBTC': 1, 'ETHBTC': 2, 'ETCBTC': 3, 'ETHLTC': 490569, 'ETCLTC': 490588, 'BTCUSDT': 490589, 'LTCUSDT': 490590, 'ETHUSDT': 490591, 'ETCUSDT': 490592, 'XMRUSDT': 852382, 'XMRBTC': 852383, 'XMRLTC': 852384, 'ZECUSDT': 1111976, 'ZECBTC': 1111977, 'ZECLTC': 1111978, 'BTCCAD': 1801843, 'LTCCAD': 1801844, 'ETHCAD': 1801845, 'ZECCAD': 1885998, 'BTCXSGD': 2372505, 'LTCXSGD': 2372506, 'ETHXSGD': 2372507, 'USDTXSGD': 2372508, 'DAIXSGD': 2372510, 'ETCXSGD': 2372511, 'BTCXIDR': 2372512}


In [7]:
# Initialize a dictionary to store the counts and amounts
transaction_details = {}

# Iterate over the transactions
for transaction in data1:
    inst_id = transaction['matchedOrder']['instId']
    pair = next((pair_name for pair_name, id_value in pair_dic.items() if id_value == inst_id), None)
    
    if pair:
        # Split the currency pair
        if len(pair) == 6:
            first_currency, second_currency = pair[:3], pair[3:]
        elif len(pair) == 7:
            first_currency, second_currency = pair[:3], pair[3:]
        elif len(pair) == 8:
            first_currency, second_currency = pair[:4], pair[4:]

        # Calculate the amounts
        first_currency_amount = float(transaction['fillQty'])
        second_currency_amount = float(transaction['fillPrice']) * float(transaction['fillQty'])

        # If the pair is not yet in the dictionary, add it with the count and amounts
        if pair not in transaction_details:
            transaction_details[pair] = {'Count': 1, first_currency: first_currency_amount, second_currency: second_currency_amount}
        else:
            # If the pair is already in the dictionary, increment the count and add the amounts
            transaction_details[pair]['Count'] += 1
            transaction_details[pair][first_currency] += first_currency_amount
            transaction_details[pair][second_currency] += second_currency_amount

# Print the transaction details
for pair, details in transaction_details.items():
    # Get the currency names from the pair
    if len(pair) == 6:
        first_currency, second_currency = pair[:3], pair[3:]
    elif len(pair) == 7:
        first_currency, second_currency = pair[:3], pair[3:]
    elif len(pair) == 8:
        first_currency, second_currency = pair[:4], pair[4:]

    print(f'Pair: {pair}, Count: {details["Count"]}, {first_currency}: {details[first_currency]:.2f}, {second_currency}: {details[second_currency]:.2f}')


Pair: LTCBTC, Count: 1241, LTC: 739.19, BTC: 2.24
Pair: LTCUSDT, Count: 63237, LTC: 23915.26, USDT: 1569842.06
Pair: ETHXSGD, Count: 1323, ETH: 331.84, XSGD: 859274.85
Pair: BTCXSGD, Count: 2778, BTC: 36.08, XSGD: 1340458.01
Pair: ZECLTC, Count: 1170, ZEC: 6.50, LTC: 7.89
Pair: ETHLTC, Count: 782, ETH: 2.11, LTC: 48.66
Pair: ETCLTC, Count: 11890, ETC: 283.93, LTC: 82.32
Pair: XMRUSDT, Count: 3375, XMR: 216.16, USDT: 41943.87
Pair: USDTXSGD, Count: 4522, USDT: 3718059.37, XSGD: 5056328.39
Pair: LTCXSGD, Count: 779, LTC: 1787.02, XSGD: 203744.84
Pair: ETHUSDT, Count: 5122, ETH: 318.73, USDT: 865657.62
Pair: ZECUSDT, Count: 2111, ZEC: 283.12, USDT: 34812.82
Pair: ETCUSDT, Count: 3340, ETC: 3211.60, USDT: 81117.41
Pair: BTCUSDT, Count: 3844, BTC: 31.02, USDT: 746222.88
Pair: ETHBTC, Count: 124, ETH: 18.27, BTC: 1.26
Pair: DAIXSGD, Count: 133, DAI: 32348.92, XSGD: 43986.34
Pair: ETCXSGD, Count: 381, ETC: 6705.82, XSGD: 330100.90
Pair: XMRLTC, Count: 403, XMR: 3.90, LTC: 9.69
Pair: XMRBTC, C